# Result Arrangement

In [1]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re


import numpy as np
import matplotlib.pyplot as plt

In [2]:
import Basic_Module as bm

## Category 

In [3]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## Path

In [4]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'
outpath = './outcome/'

In [5]:
outcomeFileList = glob(outpath+'outcome_news_classification'+'*')

In [15]:
for outcomeFile in outcomeFileList:
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    modelIs = bm.ExtractModelType(outcomeFile)
    #print (fileName)
    #print (modelIs)

In [16]:
df#.loc[:,['LogisticRegression','NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier','SVC', 'XGBoost']]

,id,title,extracted_keywords,LogisticRegression,NeuralNetwork_1,NeuralNetwork_2,RandomForestClassifier,SVC,XGBoost
0,5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'연기', '지역', '남성들', '공장', '부작용', '세보 효과', '건강'}",스포츠,사회,사회,사회,사회,경제
1,5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'자세', '식후', '커피', '디스크', '철분', '점심 식사', '입냄새'...",스포츠,사회,사회,사회,사회,경제
2,5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '앙심', '남편', '지인', '아들', '성관계', '철원', '진...",스포츠,사회,사회,사회,사회,경제
3,5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'매장', '혐의', '네이처리퍼블릭', '검찰', '롯데', '징역', '신영자...",스포츠,사회,사회,사회,사회,경제
4,5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'쪽지', '경찰', '과자', '유치장', '폭행', '혐의', '남자친구', ...",스포츠,사회,사회,사회,사회,경제
5,5a2a61bf588c13481c229d23,"화성서 발견된 포탄 모양 둥근 물체, 정체는?","{'모양 물체', '사진', '공개', '암석', '학자', 'NASA', '화성'}",스포츠,사회,사회,사회,사회,경제
6,5a2a61bf588c13481c229d24,"""음주로 '필름' 끊긴 채 성폭행, 심신장애·감형사유 아냐""","{'블랙아웃', '감형', '성폭행', '음주', '심신 장애', '범행'}",스포츠,사회,사회,사회,사회,경제
7,5a2a61bf588c13481c229d25,"최승호 PD, 해직 1997일 만에 MBC 신임 사장 내정","{'이사', '대표', '주주총회', '전국언론노동조합', '최승호', '면접', ...",스포츠,사회,사회,사회,사회,경제
8,5a2a61bf588c13481c229d26,[단독] '제2의 박찬주'? 요리사에 손톱·머리 손질 맡긴 대사 부인,"{'대사', '횡포', '외교부', '공관', '갑질', '감사', '근무', '부...",스포츠,사회,사회,사회,사회,경제
9,5a2a61bf588c13481c229d27,"김어준 ""지금도 댓글부대 운영돼..'옵션열기' 검색해봐라""","{'댓글부대 운영', '검색', '옵션 열기'}",스포츠,사회,사회,사회,사회,경제


In [ ]:
x.group()